In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting
import cvxpy as cp
from pypfopt.exceptions import OptimizationError

# Load data
emissions_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Emissions Data_66 stocks.csv")
returns_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Market Price_66 stocks.csv", index_col='Date', parse_dates=True, usecols=range(67))
sector_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\sector.csv")

# Data Cleansing
emissions_data.columns = emissions_data.columns.str.strip()
emissions_data = emissions_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'].str.replace(',', '').astype(float)
emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'] = emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'].str.replace(',', '').astype(float)

# Calculate total emissions for each instrument
emissions_data['Total Emissions'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] + emissions_data['CO2 Equivalent Emissions Indirect, Scope 2']
relevant_emissions = emissions_data.set_index('Instrument')['Total Emissions']

# Ensure the tickers in returns_data match those in relevant_emissions
tickers = returns_data.columns.intersection(relevant_emissions.index)

# Filter returns_data and relevant_emissions to include only matching tickers
returns_data_filtered = returns_data[tickers]
relevant_emissions_filtered = relevant_emissions[tickers]

# Calculate mean historical returns and sample covariance matrix
mu = expected_returns.mean_historical_return(returns_data_filtered, frequency=12, returns_data=True)
S = risk_models.risk_matrix(returns_data_filtered, method='ledoit_wolf', returns_data=True)

In [64]:
print(mu) #Mean Historical returns

AA     0.165041
AEO    0.072326
AIZ    0.136892
AKR   -0.053175
AMP    0.283265
         ...   
WHR   -0.001174
WOR    0.248323
WSR    0.063667
XHR   -0.048077
XOM    0.163286
Length: 66, dtype: float64


In [65]:
print(S) #Sample Covariance Matrix

           AA       AEO       AIZ       AKR       AMP       APH       APO  \
AA   8.649444  2.608937  0.820894  3.586521  2.427404  2.102477  2.015700   
AEO  2.608937  4.787959  0.622959  2.148176  1.448149  1.028997  1.524838   
AIZ  0.820894  0.622959  1.303265  0.439531  0.371896  0.305162  0.335225   
AKR  3.586521  2.148176  0.439531  3.522731  1.436860  1.083036  1.515414   
AMP  2.427404  1.448149  0.371896  1.436860  2.191635  1.072006  1.500877   
..        ...       ...       ...       ...       ...       ...       ...   
WHR  2.501548  1.498721  0.597294  1.212828  1.315306  1.250646  1.221500   
WOR  2.707792  1.910117  0.303637  1.494845  1.416197  1.144124  1.173015   
WSR  3.304596  1.406121  0.538150  2.183253  1.263198  0.977856  1.055865   
XHR  3.624285  2.086911  0.472853  2.951096  1.531916  1.099141  1.413029   
XOM  2.083039  0.925740  0.114781  1.445982  1.089304  0.685072  1.222861   

          AVB       AWR       AXP  ...       THS       TYL        UE  \
AA 

In [66]:
#Optimize for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
weights = ef.max_sharpe()

In [67]:
# Convert weights to a DataFrame
tangency_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])

# Ensure the index is named 'Tickers'
tangency_weights.index.name = 'Tickers'

# Filter and sort the weights
filtered_weights = tangency_weights[tangency_weights['Weight'] > 0].sort_values(by='Weight', ascending=False)

# Reset the index to convert the index into a column
filtered_weights = filtered_weights.reset_index()

# Rename the columns
filtered_weights.columns = ['Tickers', 'Weight']

# Save the DataFrame to a CSV file
filtered_weights.to_csv('tangency_weights.csv', index=False)

In [68]:
#Calculate tangency portfolio returns
tangency_returns = (tangency_weights.T.values*returns_data).sum(axis=1)
print(tangency_returns)

Date
2019-06-01    0.085919
2019-07-01   -0.024340
2019-08-01    0.044941
2019-09-01   -0.006753
2019-10-01    0.033331
2019-11-01    0.024436
2019-12-01   -0.008887
2020-01-01    0.009473
2020-02-01   -0.063588
2020-03-01   -0.055932
2020-04-01    0.080577
2020-05-01    0.147482
2020-06-01    0.036470
2020-07-01    0.024030
2020-08-01    0.082817
2020-09-01   -0.032907
2020-10-01   -0.011245
2020-11-01    0.155068
2020-12-01    0.019044
2021-01-01    0.009201
2021-02-01    0.021131
2021-03-01    0.066145
2021-04-01    0.020547
2021-05-01    0.028610
2021-06-01    0.036616
2021-07-01    0.019762
2021-08-01    0.036571
2021-09-01   -0.023959
2021-10-01    0.064320
2021-11-01    0.002713
2021-12-01    0.029966
2022-01-01   -0.058578
2022-02-01    0.038413
2022-03-01    0.036750
2022-04-01   -0.078124
2022-05-01    0.011008
2022-06-01   -0.015492
2022-07-01    0.100709
2022-08-01    0.016655
2022-09-01   -0.075477
2022-10-01    0.108479
2022-11-01    0.024443
2022-12-01   -0.056180
2023-0

In [69]:
print(tangency_weights)

           Weight
Tickers          
AA       0.000000
AEO      0.000000
AIZ      0.041829
AKR      0.000000
AMP      0.000000
...           ...
WHR      0.000000
WOR      0.000000
WSR      0.000000
XHR      0.000000
XOM      0.000000

[66 rows x 1 columns]


In [70]:
#Calculating performance metrics 

mu_tangency_monthly = tangency_returns.mean()
mu_tangency_annual = (1+mu_tangency_monthly)**12-1

sigma_sq_tangency = tangency_returns.var(ddof=1)
sigma_tangency_monthly = tangency_returns.std(ddof=1)
sigma_tangency_annual = sigma_tangency_monthly*np.sqrt(12)

sr_tangency = mu_tangency_monthly/sigma_tangency_monthly*np.sqrt(12)

num_inv_tangency = np.sum(tangency_weights.values>0)
num_sectors_tangency = sector_data.merge(tangency_weights.loc[tangency_weights['Weight']>0], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

In [71]:
print(f"mu_tangency_annual: {mu_tangency_annual}")
print(f"sigma_tangency_annual: {sigma_tangency_annual}")
print(f"sr_tangency: {sr_tangency}")
print(f"num_inv_tangency: {num_inv_tangency}")
print(f"num_sectors_tangency: {num_sectors_tangency}")

mu_tangency_annual: 0.33065086313143044
sigma_tangency_annual: 0.17372802365015497
sr_tangency: 1.6640700425700523
num_inv_tangency: 11
num_sectors_tangency: 6


In [75]:
tangency_sector_weights = tangency_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()
print("Tangency Sector Weights:\n", tangency_sector_weights)

Tangency Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.000000
Consumer Cyclicals        0.282539
Consumer Non-Cyclicals    0.117865
Energy                    0.024224
Financials                0.095747
Healthcare                0.000000
Industrials               0.139074
Real Estate               0.000000
Technology                0.340552
Utilities                 0.000000
Name: Weight, dtype: float64


In [ ]:
#### Raw Emissions Tangency based Portfolio 

In [210]:
import pandas as pd

# Load the data
emissions_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Emissions Data_66 stocks.csv")
returns_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Market Price_66 stocks.csv", index_col='Date', parse_dates=True, usecols=range(67))
weights_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\tgncy_weights.csv")  # Load the weights from the generated CSV

# Strip leading and trailing spaces from the column names of emissions_data
emissions_data.columns = emissions_data.columns.str.strip()

# Strip leading and trailing spaces from the 'Instrument' column values
emissions_data['Instrument'] = emissions_data['Instrument'].str.strip()

# Set the index for easy access to rows based on Instrument
emissions_data.set_index('Instrument', inplace=True)

# Ensure the correct column names in weights_data
weights_data.columns = ['Tickers', 'Weight']

# Convert weights to a Series for easy manipulation
weights = pd.Series(weights_data.set_index('Tickers')['Weight'])

# Ensure matching data types for the indices
returns_data.columns = returns_data.columns.astype(str)
emissions_data.index = emissions_data.index.astype(str)
weights.index = weights.index.astype(str)

# Filter emissions data to match the tickers in returns_data
direct_emissions = emissions_data.loc[returns_data.columns, 'CO2 Equivalent Emissions Direct, Scope 1']
indirect_emissions = emissions_data.loc[returns_data.columns, 'CO2 Equivalent Emissions Indirect, Scope 2']

# Remove commas and convert emissions data to float to ensure correct data types
direct_emissions = direct_emissions.str.replace(',', '').astype(float)
indirect_emissions = indirect_emissions.str.replace(',', '').astype(float)

# Calculate the total emissions for each ticker
total_emissions = direct_emissions + indirect_emissions

# Ensure weights are float
weights = weights.astype(float)

# Calculate the weighted emissions for the portfolio
# Reindex weights to match total_emissions index
weights_reindexed = weights.reindex(total_emissions.index).fillna(0)

# Ensure weights_reindexed and total_emissions are float
weights_reindexed = weights_reindexed.astype(float)
total_emissions = total_emissions.astype(float)

# Perform the multiplication
weighted_emissions = weights_reindexed * total_emissions

# Calculate the sum of weighted emissions to get the 'Raw Emission of Market Cap Based Portfolio'
raw_emission_tangency_based_portfolio = weighted_emissions.sum()

# Create a DataFrame to store the results and save to CSV
emission_results = pd.DataFrame({
    'Ticker': total_emissions.index,
    'Weight': weights_reindexed,
    'Direct Emissions': direct_emissions,
    'Indirect Emissions': indirect_emissions,
    'Total Emissions': total_emissions,
    'Weighted Emissions': weighted_emissions
})

emission_results.to_csv('raw_emission_tangency_based_portfolio.csv', index=False)

In [212]:
# Assuming 'weights' contains the tangency portfolio weights
# Ensure weights are correctly aligned with returns_data columns
tangency_weights = pd.Series(weights, index=returns_data.columns).reindex(returns_data.columns, fill_value=0).values

# Calculate the tangency portfolio returns
tangency_returns = (returns_data * tangency_weights).sum(axis=1)

# Calculate the monthly and annual mean returns for the tangency portfolio
mu_tangency_monthly = tangency_returns.mean()
mu_tangency_annual = (1 + mu_tangency_monthly) ** 12 - 1

# Calculate the monthly and annual volatility (standard deviation) for the tangency portfolio
sigma_tangency_monthly = tangency_returns.std(ddof=1)
sigma_tangency_annual = sigma_tangency_monthly * np.sqrt(12)

# Calculate the Sharpe Ratio for the tangency portfolio (assuming risk-free rate is 0)
sr_tangency = mu_tangency_monthly / sigma_tangency_monthly * np.sqrt(12)

# Count the number of investments and sectors in the tangency portfolio
num_inv_tangency = np.sum(tangency_weights > 0)
num_sectors_tangency = len(np.unique(sector_data.loc[sector_data['Instrument'].isin(returns_data.columns[tangency_weights > 0]), 'TRBC Economic Sector Name']))

# Print the top 15 weights for the tangency portfolio
top15_weights = pd.Series(tangency_weights, index=returns_data.columns).sort_values(ascending=False).head(15)
print("Top 15 Weights:\n", top15_weights)
print(f"Monthly Mean Return: {mu_tangency_monthly:.4f}")
print(f"Annual Mean Return: {mu_tangency_annual:.4f}")
print(f"Monthly Volatility: {sigma_tangency_monthly:.4f}")
print(f"Annual Volatility: {sigma_tangency_annual:.4f}")
print(f"Sharpe Ratio: {sr_tangency:.4f}")
print(f"Number of Investments: {num_inv_tangency}")
print(f"Number of Sectors: {num_sectors_tangency}")


Top 15 Weights:
 BJ     0.282539
BAH    0.167469
HWM    0.139074
NOW    0.088939
GE     0.071816
TYL    0.064191
APO    0.053919
HRB    0.046049
AIZ    0.041829
HES    0.024224
APH    0.019953
AA          NaN
AEO         NaN
AKR         NaN
AMP         NaN
dtype: float64
Monthly Mean Return: 0.0241
Annual Mean Return: 0.3307
Monthly Volatility: 0.0502
Annual Volatility: 0.1737
Sharpe Ratio: 1.6641
Number of Investments: 11
Number of Sectors: 6


In [208]:
# Calculate the benchmark returns (e.g., an equally weighted portfolio or a specific benchmark)
benchmark_weights = np.ones(len(returns_data.columns)) / len(returns_data.columns)
benchmark_returns = (returns_data * benchmark_weights).sum(axis=1)

# Calculate the monthly and annual mean returns for the benchmark
mu_benchmark_monthly = benchmark_returns.mean()
mu_benchmark_annual = (1 + mu_benchmark_monthly) ** 12 - 1

# Calculate the monthly and annual volatility (standard deviation) for the benchmark
sigma_benchmark_monthly = benchmark_returns.std(ddof=1)
sigma_benchmark_annual = sigma_benchmark_monthly * np.sqrt(12)

# Calculate the Sharpe Ratio for the benchmark (assuming risk-free rate is 0)
sr_benchmark = mu_benchmark_monthly / sigma_benchmark_monthly * np.sqrt(12)
                                                                        
# Define the benchmark emissions (e.g., an equally weighted portfolio or a specific benchmark)
benchmark_emissions = (total_emissions * benchmark_weights).sum()

# Count the number of investments and sectors in the benchmark
num_inv_benchmark = np.sum(benchmark_weights > 0)
num_sectors_benchmark = returns_data.columns.nunique()

# Print the top 15 weights (though for an equally weighted portfolio, all weights are the same)                                                                  
print("Top 15 Weights:\n", benchmark_weights)
print(f"Monthly Mean Return: {mu_benchmark_monthly}")
print(f"Annual Mean Return: {mu_benchmark_annual}")
print(f"Monthly Volatility: {sigma_benchmark_monthly}")
print(f"Annual Volatility: {sigma_benchmark_annual}")
print(f"Sharpe Ratio: {sr_benchmark}")
print(f"Number of Investments: {num_inv_benchmark}")
print(f"Number of Sectors: {num_sectors_benchmark}")

Top 15 Weights:
 [0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152]
Monthly Mean Return: 0.014277600506310523
Annual Mean Return: 0.1854466524986762
Monthly Volatility: 0.06908643322599198
Annual Volatility: 0.23932242492226544
Sharpe Ratio: 0.7159011786353766
Number of Investments: 66
Number of Sectors: 66


In [ ]:
#### Constrained Portfolio - 10% Reduced Emissions 

In [188]:
# Ensure Carbon Intensity values are strings, remove commas, and convert to float
emissions_data['Carbon Intensity'] = emissions_data['Carbon Intensity'].astype(str).str.replace(',', '').astype(float)

# Define constraints
def TE(w, portfolio_returns, benchmark_returns):
    xi = (w @ portfolio_returns.T - benchmark_returns.values.reshape(-1))
    mean = cp.sum(xi) / len(benchmark_returns)
    return cp.sum_squares(xi - mean)

def CI(w, portfolio_intensity):
    xi = w @ portfolio_intensity.T
    carbon_intensity = cp.sum(xi)
    return carbon_intensity

ef_constrained1 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained1.add_constraint(lambda w: TE(w, returns_data, benchmark_returns) <= 0.1**2)
ef_constrained1.add_constraint(lambda w: CI(w, emissions_data['Carbon Intensity']) <= 0.9 * benchmark_emissions)
weights = ef_constrained1.convex_objective(TE, portfolio_returns=returns_data, benchmark_returns=benchmark_returns)

#Calculate the total emissionsfor the constrained portfolio
constrained1_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained1_weights[constrained1_weights['Weight']>0].sort_values(by='Weight', ascending=False)

constrained1_returns = (constrained1_weights.T.values*returns_data).sum(axis=1)

mu_constrained1_monthly = constrained1_returns.mean()
mu_constrained1_annual = (1+mu_constrained1_monthly)**12-1

sigma_sq_constrained1 = constrained1_returns.var(ddof=1)
sigma_constrained1_monthly = constrained1_returns.std(ddof=1)
sigma_constrained1_annual = sigma_constrained1_monthly*np.sqrt(12)

sr_constrained1 = mu_constrained1_monthly/sigma_constrained1_monthly*np.sqrt(12)

constrained1_te = (constrained1_returns - benchmark_returns.T).values.std(ddof=1)

num_inv_constrained1 = np.sum(constrained1_weights.values>10**-4)
num_sectors_constrained1 = df_sector.merge(constrained1_weights.loc[constrained1_weights['Weight']>10**-4], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

#Calculate the emissions for the constrained portfolio
constrained1_emissions = (constrained1_weights['Weight'].values * total_emissions.values).sum()

c1_sector_weights = constrained1_weights.merge(df_sector, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print("Top 15 Weights:\n", constrained1_weights)
print(f"Monthly Mean Return: {mu_constrained1_monthly}")
print(f"Annual Mean Return: {mu_constrained1_annual}")
print(f"Monthly Volatility: {sigma_constrained1_monthly}")
print(f"Annual Volatility: {sigma_constrained1_annual}")
print(f"Sharpe Ratio: {sr_constrained1}")
print(f"Tracking Error: {constrained1_te}")
print(f"Number of Investments: {num_inv_constrained1}")
print(f"Number of Sectors: {num_sectors_constrained1}")
print(f"Total Emissions: {constrained1_emissions}")
print("Sector Weights:\n", c1_sector_weights)

Top 15 Weights:
        Weight
AA   0.008504
AEO  0.014865
AIZ  0.014821
AKR  0.031128
AMP  0.029059
..        ...
WHR  0.009298
WOR  0.017149
WSR  0.013674
XHR  0.016906
XOM  0.004262

[66 rows x 1 columns]
Monthly Mean Return: 0.013868277790965658
Annual Mean Return: 0.17971857946975112
Monthly Volatility: 0.06908642695839595
Annual Volatility: 0.2393224032106759
Sharpe Ratio: 0.6953771617657067
Tracking Error: 1.7521689412677686e-07
Number of Investments: 66
Number of Sectors: 3
Total Emissions: 3306780.232384445
Sector Weights:
 TRBC Economic Sector Name
Sector A    0.363301
Sector B    0.292814
Sector C    0.343885
Name: Weight, dtype: float64


In [105]:
#### Constrained Portfolio - 20% Carbon Intensity Reduction 

In [185]:
ef_constrained2 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained2.add_constraint(lambda w: TE(w, returns_data, benchmark_returns) <= 0.1**2)
ef_constrained2.add_constraint(lambda w: CI(w, emissions_data['Carbon Intensity']) <= 0.8*benchmark_emissions)
weights = ef_constrained2.convex_objective(TE, portfolio_returns= returns_data, benchmark_returns=benchmark_returns)
constrained2_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained2_weights[constrained2_weights['Weight']>10**-6].sort_values(by='Weight', ascending=False)

constrained2_returns = (constrained2_weights.T.values*returns_data).sum(axis=1)

mu_constrained2_monthly = constrained2_returns.mean()
mu_constrained2_annual = (1+mu_constrained2_monthly)**12-1

sigma_sq_constrained2 =constrained2_returns.var(ddof=1)
sigma_constrained2_monthly = constrained2_returns.std(ddof=1)
sigma_constrained2_annual = sigma_constrained2_monthly*np.sqrt(12)

sr_constrained2 = mu_constrained2_monthly/sigma_constrained2_monthly*np.sqrt(12)

constrained2_te = (constrained2_returns - benchmark_returns.T).values.std(ddof=1)

num_inv_constrained2 = np.sum(constrained2_weights.values>10**-6)
num_sectors_constrained2 = df_sector.merge(constrained2_weights.loc[constrained2_weights['Weight']>10**-6], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()
constrained2_emissions = (constrained2_weights['Weight'].values * total_emissions.values).sum()

c2_sector_weights = constrained2_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print("Top 15 Weights:\n", constrained2_weights)
print(f"Monthly Mean Return: {mu_constrained2_monthly}")
print(f"Annual Mean Return: {mu_constrained2_annual}")
print(f"Monthly Volatility: {sigma_constrained2_monthly}")
print(f"Annual Volatility: {sigma_constrained2_annual}")
print(f"Sharpe Ratio: {sr_constrained2}")
print(f"Tracking Error: {constrained2_te}")
print(f"Number of Investments: {num_inv_constrained2}")
print(f"Number of Sectors: {num_sectors_constrained2}")
print(f"Total Emissions: {constrained2_emissions}")
print("Sector Weights:\n", c2_sector_weights)

Top 15 Weights:
        Weight
AA   0.006374
AEO  0.013503
AIZ  0.016544
AKR  0.036580
AMP  0.036016
..        ...
WHR  0.006722
WOR  0.019456
WSR  0.011017
XHR  0.018446
XOM  0.001206

[66 rows x 1 columns]
Monthly Mean Return: 0.013693019957461776
Annual Mean Return: 0.17727378298420704
Monthly Volatility: 0.0690864327542376
Annual Volatility: 0.23932242328806036
Sharpe Ratio: 0.686589401995826
Tracking Error: 1.9377843059993325e-08
Number of Investments: 66
Number of Sectors: 3
Total Emissions: 2901376.7532237875
Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.056291
Consumer Cyclicals        0.174689
Consumer Non-Cyclicals    0.115367
Energy                    0.051382
Financials                0.145208
Healthcare                0.031039
Industrials               0.091394
Real Estate               0.141192
Technology                0.071138
Utilities                 0.122299
Name: Weight, dtype: float64


In [113]:
#### Constrained Portfolio - 50% Reduction in Carbon Intensity

In [186]:
ef_constrained3 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained3.add_constraint(lambda w: TE(w, returns_data, benchmark_returns) <= 0.1**2)
ef_constrained3.add_constraint(lambda w: CI(w, emissions_data['Carbon Intensity']) <= 0.5*benchmark_emissions)
weights = ef_constrained3.convex_objective(TE, portfolio_returns=returns_data, benchmark_returns=benchmark_returns)

constrained3_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained3_weights[constrained3_weights['Weight']>10**-4].sort_values(by='Weight', ascending=False).head(15)
# Ensure all relevant columns are numeric
emissions_data = emissions_data.apply(lambda x: x.str.replace(',', '').astype(float) if x.dtype == 'object' else x)

# Calculate the returns for the constrained portfolio
constrained3_returns = (constrained3_weights['Weight'].values * returns_data).sum(axis=1)

# Calculate the monthly and annual mean returns
mu_constrained3_monthly = constrained3_returns.mean()
mu_constrained3_annual = (1 + mu_constrained3_monthly) ** 12 - 1

# Calculate the monthly and annual standard deviation (volatility)
sigma_sq_constrained3 = constrained3_returns.var(ddof=1)
sigma_constrained3_monthly = constrained3_returns.std(ddof=1)
sigma_constrained3_annual = sigma_constrained3_monthly * np.sqrt(12)

# Calculate the Sharpe Ratio
sr_constrained3 = mu_constrained3_monthly / sigma_constrained3_monthly * np.sqrt(12)

# Calculate the tracking error
constrained3_te = (constrained3_returns - benchmark_returns.values).std(ddof=1)

# Count the number of investments and sectors in the constrained portfolio
num_inv_constrained3 = np.sum(constrained3_weights['Weight'].values > 10**-4)
num_sectors_constrained3 = sector_data.merge(constrained3_weights[constrained3_weights['Weight'] > 10**-4], 
                                           left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

# Calculate the total emissions for the constrained portfolio
constrained3_emissions = (constrained3_weights['Weight'].values * emissions_data.sum(axis=1).values).sum()

c3_sector_weights = constrained3_weights.merge(df_sector, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print("Top 15 Weights:\n", constrained3_weights)
print(f"Monthly Mean Return: {mu_constrained3_monthly}")
print(f"Annual Mean Return: {mu_constrained3_annual}")
print(f"Monthly Volatility: {sigma_constrained3_monthly}")
print(f"Annual Volatility: {sigma_constrained3_annual}")
print(f"Sharpe Ratio: {sr_constrained3}")
print(f"Tracking Error: {constrained3_te}")
print(f"Number of Investments: {num_inv_constrained3}")
print(f"Number of Sectors: {num_sectors_constrained3}")
print(f"Total Emissions: {constrained3_emissions}")
print("Sector Weights:\n", c3_sector_weights)

Top 15 Weights:
            Weight
AA   2.241386e-03
AEO  6.270508e-03
AIZ  2.009310e-02
AKR  4.279344e-02
AMP  4.735579e-02
..            ...
WHR  5.550544e-03
WOR  2.037656e-02
WSR  1.333657e-02
XHR  1.640618e-02
XOM  6.743303e-07

[66 rows x 1 columns]
Monthly Mean Return: 0.013673610616173759
Annual Mean Return: 0.17700331409056735
Monthly Volatility: 0.06908479475522827
Annual Volatility: 0.23931674909304654
Sharpe Ratio: 0.6856324432615846
Tracking Error: 0.00018642680628404594
Number of Investments: 58
Number of Sectors: 10
Total Emissions: 26692875363.1095
Sector Weights:
 TRBC Economic Sector Name
Sector A    0.330820
Sector B    0.282942
Sector C    0.386238
Name: Weight, dtype: float64


In [ ]:
#### Contrained Portfolio - Tangency Portfolio with 20% less Carbon Intensity

In [187]:
import pandas as pd
import numpy as np
from pypfopt import expected_returns, risk_models

# Load data
emissions_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Emissions Data_66 stocks.csv")
returns_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Market Price_66 stocks.csv", index_col='Date', parse_dates=True, usecols=range(67))
sector_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\sector.csv")

# Data Cleansing
emissions_data.columns = emissions_data.columns.str.strip()
emissions_data = emissions_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'].str.replace(',', '').astype(float)
emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'] = emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'].str.replace(',', '').astype(float)

# Calculate total emissions for each instrument
emissions_data['Total Emissions'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] + emissions_data['CO2 Equivalent Emissions Indirect, Scope 2']

# Convert Carbon Intensity to numeric
emissions_data['Carbon Intensity'] = emissions_data['Carbon Intensity'].str.replace(',', '').astype(float)

# Ensure the tickers in returns_data match those in emissions_data
tickers = returns_data.columns.intersection(emissions_data['Instrument'])

# Filter returns_data and emissions_data to include only matching tickers
returns_data_filtered = returns_data[tickers]
emissions_data_filtered = emissions_data.set_index('Instrument').loc[tickers]

# Calculate mean historical returns and sample covariance matrix
mu = expected_returns.mean_historical_return(returns_data_filtered, frequency=12, returns_data=True)
S = risk_models.risk_matrix(returns_data_filtered, method='ledoit_wolf', returns_data=True)

# Define constraints
def calculate_portfolio_return(weights, mu):
    return np.dot(weights, mu)

def calculate_portfolio_volatility(weights, S):
    return np.sqrt(np.dot(weights.T, np.dot(S, weights)))

def calculate_sharpe_ratio(weights, mu, S, risk_free_rate=0.02):
    portfolio_return = calculate_portfolio_return(weights, mu)
    portfolio_volatility = calculate_portfolio_volatility(weights, S)
    return (portfolio_return - risk_free_rate) / portfolio_volatility

def calculate_carbon_intensity(weights, carbon_intensity):
    return np.dot(weights, carbon_intensity)

# Heuristic optimization
n_assets = len(mu)
best_sharpe_ratio = -np.inf
best_weights = np.zeros(n_assets)
carbon_intensity_threshold = 0.8 * emissions_data_filtered['Carbon Intensity'].mean()

for _ in range(10000):  # Number of iterations
    weights = np.random.random(n_assets)
    weights /= np.sum(weights)  # Normalize weights to sum to 1
    
    portfolio_sharpe_ratio = calculate_sharpe_ratio(weights, mu, S)
    portfolio_carbon_intensity = calculate_carbon_intensity(weights, emissions_data_filtered['Carbon Intensity'].values)
    
    if portfolio_carbon_intensity <= carbon_intensity_threshold and portfolio_sharpe_ratio > best_sharpe_ratio:
        best_sharpe_ratio = portfolio_sharpe_ratio
        best_weights = weights

# Convert weights to a DataFrame
constrained4_weights = pd.DataFrame(best_weights, index=returns_data_filtered.columns, columns=['Weight'])
top15_constrained4_weights = constrained4_weights[constrained4_weights['Weight'] > 10**-4].sort_values(by='Weight', ascending=False).head(15)
print("Top 15 Weights:\n", top15_constrained4_weights)

# Calculate returns for the constrained portfolio
constrained4_returns = (constrained4_weights.T.values * returns_data_filtered).sum(axis=1)

# Calculate monthly and annual mean returns
mu_constrained4_monthly = constrained4_returns.mean()
mu_constrained4_annual = (1 + mu_constrained4_monthly) ** 12 - 1

# Calculate monthly and annual standard deviation (volatility)
sigma_sq_constrained4 = constrained4_returns.var(ddof=1)
sigma_constrained4_monthly = constrained4_returns.std(ddof=1)
sigma_constrained4_annual = sigma_constrained4_monthly * np.sqrt(12)

# Calculate Sharpe Ratio
sr_constrained4 = mu_constrained4_monthly / sigma_constrained4_monthly * np.sqrt(12)

# Calculate tracking error
benchmark_returns = returns_data_filtered.mean(axis=1)  # Assuming benchmark returns are the average returns
constrained4_te = (constrained4_returns - benchmark_returns.values).std(ddof=1)

# Count the number of investments and sectors in the constrained portfolio
num_inv_constrained4 = np.sum(constrained4_weights['Weight'].values > 10**-4)
num_sectors_constrained4 = sector_data.merge(constrained4_weights[constrained4_weights['Weight'] > 10**-4], 
                                             left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

# Calculate the total emissions for the constrained portfolio
constrained4_emissions = (constrained4_weights['Weight'].values * emissions_data_filtered['Total Emissions'].values).sum()

# Calculate sector weights
c4_sector_weights = constrained4_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

# Print the results
print(f"mu_constrained4_annual: {mu_constrained4_annual}")
print(f"sigma_constrained4_annual: {sigma_constrained4_annual}")
print(f"sr_constrained4: {sr_constrained4}")
print(f"constrained4_te: {constrained4_te}")
print(f"num_inv_constrained4: {num_inv_constrained4}")
print(f"num_sectors_constrained4: {num_sectors_constrained4}")
print(f"constrained4_emissions: {constrained4_emissions}")
print("Sector Weights:\n", c4_sector_weights)


Top 15 Weights:
        Weight
NEE  0.032275
BJ   0.032101
UE   0.031460
DOC  0.031173
OXY  0.030709
GE   0.030330
HRB  0.030302
WOR  0.029825
IQV  0.029790
NFG  0.027333
NOW  0.026942
APO  0.026909
FDX  0.026564
AEO  0.026408
RYI  0.025764
mu_constrained4_annual: 0.2169453569007307
sigma_constrained4_annual: 0.23209487057527817
sr_constrained4: 0.852922764572389
constrained4_te: 0.006770537758857201
num_inv_constrained4: 65
num_sectors_constrained4: 10
constrained4_emissions: 3763342.7704585786
Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.058141
Consumer Cyclicals        0.172219
Consumer Non-Cyclicals    0.128328
Energy                    0.069057
Financials                0.151510
Healthcare                0.034553
Industrials               0.058908
Real Estate               0.120326
Technology                0.097508
Utilities                 0.109449
Name: Weight, dtype: float64


In [128]:
#### Constrained Portfolio - Tangency w/ 20% CI reduction & Sector Balance

In [196]:
import pandas as pd
import numpy as np
from pypfopt import EfficientFrontier, expected_returns, risk_models

# Ensure 'emissions_data' and 'sector_data' have numeric columns where expected
emissions_data = emissions_data.apply(pd.to_numeric, errors='coerce')
sector_data['Instrument'] = sector_data['Instrument'].astype(str)
returns_data.columns = returns_data.columns.astype(str)

# Assuming benchmark_weights is a numpy array
benchmark_weights_df = pd.DataFrame(benchmark_weights, index=returns_data.columns, columns=['Weight'])

# Merge with sector data
sector_weights = benchmark_weights_df.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

lower_bound = (sector_weights * 0.7).to_dict()
upper_bound = (sector_weights * 1.3).to_dict()

ef_constrained5 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained5.add_constraint(lambda w: CI(w, emissions_data['Carbon Intensity']) <= 0.8 * benchmark_emissions)
ef_constrained5.add_sector_constraints(sector_mapper, lower_bound, upper_bound)
weights = ef_constrained5.max_sharpe()

constrained5_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained5_returns = (constrained5_weights.T.values * returns_data.values).sum(axis=1)

mu_constrained5_monthly = constrained5_returns.mean()
mu_constrained5_annual = (1 + mu_constrained5_monthly) ** 12 - 1

sigma_sq_constrained5 = constrained5_returns.var(ddof=1)
sigma_constrained5_monthly = constrained5_returns.std(ddof=1)
sigma_constrained5_annual = sigma_constrained5_monthly * np.sqrt(12)

sr_constrained5 = mu_constrained5_monthly / sigma_constrained5_monthly * np.sqrt(12)

constrained5_te = (constrained5_returns - tangency_returns).std(ddof=1)

num_inv_constrained5 = np.sum(constrained5_weights.values > 0)
num_sectors_constrained5 = sector_data.merge(constrained5_weights[constrained5_weights['Weight'] > 0], 
                                             left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

constrained5_emissions = (constrained5_weights['Weight'].values * emissions_data.sum(axis=1).values).sum()
c5_sector_weights = constrained5_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print(f"Constrained Portfolio Weights:\n{constrained5_weights}")
print(f"Monthly Mean Return: {mu_constrained5_monthly}")
print(f"Annual Mean Return: {mu_constrained5_annual}")
print(f"Monthly Volatility: {sigma_constrained5_monthly}")
print(f"Annual Volatility: {sigma_constrained5_annual}")
print(f"Sharpe Ratio: {sr_constrained5}")
print(f"Tracking Error: {constrained5_te}")
print(f"Number of Investments: {num_inv_constrained5}")
print(f"Number of Sectors: {num_sectors_constrained5}")
print(f"Total Emissions: {constrained5_emissions}")
print(f"Sector Weights:\n{c5_sector_weights}")


Constrained Portfolio Weights:
       Weight
AA   0.000000
AEO  0.000000
AIZ  0.056533
AKR  0.000000
AMP  0.000000
..        ...
WHR  0.000000
WOR  0.042424
WSR  0.004429
XHR  0.000000
XOM  0.000000

[66 rows x 1 columns]
Monthly Mean Return: 0.021762686160174738
Annual Mean Return: 0.29479335881890556
Monthly Volatility: 0.05108626601909898
Annual Volatility: 0.17696801662811776
Sharpe Ratio: 1.4757030049722746
Tracking Error: 0.011280802996756295
Number of Investments: 14
Number of Sectors: 10
Total Emissions: 7959434.731711317
Sector Weights:
TRBC Economic Sector Name
Basic Materials           0.042424
Consumer Cyclicals        0.216667
Consumer Non-Cyclicals    0.092424
Energy                    0.042424
Financials                0.157576
Healthcare                0.021212
Industrials               0.098485
Real Estate               0.106061
Technology                0.137879
Utilities                 0.084848
Name: Weight, dtype: float64


In [139]:
#### Summary Dataframe Composition

In [225]:
import pandas as pd

# Create individual DataFrames for each portfolio with the provided data
bp = pd.DataFrame({
    'Market Cap-Weighted': [mu_benchmark_monthly, mu_benchmark_annual, sigma_benchmark_monthly,
                            sigma_benchmark_annual, sr_benchmark, num_inv_benchmark, num_sectors_benchmark, '', 0]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

tp = pd.DataFrame({
    'Tangency': [mu_tangency_monthly, mu_tangency_annual, sigma_tangency_monthly,
                 sigma_tangency_annual, sr_tangency, num_inv_tangency, num_sectors_tangency, '', 0]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p1 = pd.DataFrame({
    'Market Cap-Weighted w/ 10% CI Reduction': [mu_constrained1_monthly, mu_constrained1_annual, sigma_constrained1_monthly,
                                                sigma_constrained1_annual, sr_constrained1, num_inv_constrained1, num_sectors_constrained1, 'Market-Cap', 100*constrained1_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p2 = pd.DataFrame({
    'Market Cap-Weighted w/ 20% CI Reduction': [mu_constrained2_monthly, mu_constrained2_annual, sigma_constrained2_monthly,
                                                sigma_constrained2_annual, sr_constrained2, num_inv_constrained2, num_sectors_constrained2, 'Market-Cap', 100*constrained2_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p3 = pd.DataFrame({
    'Market Cap-Weighted w/ 50% CI Reduction': [mu_constrained3_monthly, mu_constrained3_annual, sigma_constrained3_monthly,
                                                sigma_constrained3_annual, sr_constrained3, num_inv_constrained3, num_sectors_constrained3, 'Market-Cap', 100*constrained3_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p4 = pd.DataFrame({
    'Tangency w/ 20% CI Reduction': [mu_constrained4_monthly, mu_constrained4_annual, sigma_constrained4_monthly,
                                     sigma_constrained4_annual, sr_constrained4, num_inv_constrained4, num_sectors_constrained4, 'Tangency', 100*constrained4_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p5 = pd.DataFrame({
    'Tangency w/ 20% CI Reduction & Sector Balance': [mu_constrained5_monthly, mu_constrained5_annual, sigma_constrained5_monthly,
                                                      sigma_constrained5_annual, sr_constrained5, num_inv_constrained5, num_sectors_constrained5, 'Tangency', 100*constrained5_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

# Concatenate all DataFrames
df_summary = pd.concat([bp, tp, p1, p2, p3, p4, p5])

# Format the DataFrame
df_summary = df_summary.style.format({
    'Monthly Return': '{:,.4f}',
    'Annual Return': '{:,.4f}',
    'Monthly Volatility': '{:,.4f}',
    'Annual Volatility': '{:,.4f}',
    'Sharpe Ratio': '{:,.4f}',
    'Number of Investments': '{:,.0f}',
    'Number of Sectors': '{:,.0f}',
    'Tracking Error': '{:,.4f}%'
})

# Convert styled DataFrame back to regular DataFrame
df_summary_regular = df_summary.data

# Save the DataFrame to a CSV file
df_summary_regular.to_csv('df_summary.csv', index=True)

# Display the styled DataFrame
df_summary


,Monthly Return,Annual Return,Monthly Volatility,Annual Volatility,Sharpe Ratio,Number of Investments,Number of Sectors,Benchmark,Tracking Error
Market Cap-Weighted,0.0143,0.1854,0.0691,0.2393,0.7159,66,66,,0.0000%
Tangency,0.0241,0.3307,0.0502,0.1737,1.6641,11,6,,0.0000%
Market Cap-Weighted w/ 10% CI Reduction,0.0139,0.1797,0.0691,0.2393,0.6954,66,3,Market-Cap,0.0000%
Market Cap-Weighted w/ 20% CI Reduction,0.0137,0.1773,0.0691,0.2393,0.6866,66,3,Market-Cap,0.0000%
Market Cap-Weighted w/ 50% CI Reduction,0.0137,0.1770,0.0691,0.2393,0.6856,58,10,Market-Cap,0.0186%
Tangency w/ 20% CI Reduction,0.0165,0.2169,0.0670,0.2321,0.8529,65,10,Tangency,0.6771%
Tangency w/ 20% CI Reduction & Sector Balance,0.0218,0.2948,0.0511,0.1770,1.4757,14,10,Tangency,1.1281%
